# Retrieval Augmented Generation Base Custom Chatbot

This notebook contains implementation of retrieval augmented generation (RAG) based chatbot.

The chatbot is specifically designed for **conversation about the timeline of artificial intelligence (AI)** to answer questions related to events about the advancement of AI up to now.

We will be using `gpt-3.5-turbo-instruct` model, which was trained on data upto **September 2021**. Thus, the model does not have information about AI advancements prior to 2021. The RAG enables the chatbot to access and incorporate relevant and upto date information from external source that the model was not trained on. This ensures accurate and contexually appropriate response. We will be using an upto date from the Wikipedia page at [Timeline of artificial intelligence](https://en.wikipedia.org/wiki/Timeline_of_artificial_intelligence).

Here are sample question that we want to answere using the chatboot.

In [ ]:
QUESTIONS = ["What significant achievement did OpenAI's GPT-3 accomplish compared to Microsoft's Turing Natural Language Generation model?",
             "When did Google release Gemini 1.5?",
             "When did Apple announced 'Apple Intelligence' which incorporates ChatGPT into new IPhones and Siri?",
             "What was the purpose of the inaugural 'AI Insight Forum' held by the US Senate in September 2023, and who were some of the prominent attendees?",
             "What were the key developments and announcements related to AI made in February 2024 by Google and OpenAI?"
            ]

# print(f"There are {len(QUESTIONS)} QUESTIONS\n{QUESTIONS}")

## Install libraries

In [ ]:
!pip install -q openai==0.28 tiktoken beautifulsoup4 numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00


## Import required libraries

In [ ]:
import requests
from IPython.display import HTML
import re

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from openai.embeddings_utils import distances_from_embeddings, get_embedding

import tiktoken
import openai

# SET OPENAI API KEY
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-379368067126677338956666a0c6ab2b3362.58156147"

## Define variables

In [ ]:
open_ai_model_name = "gpt-3.5-turbo-instruct"
embedding_model_name = "text-embedding-ada-002"
batch_size = 64
max_tokens = 100
# url
url = "https://en.wikipedia.org/wiki/Timeline_of_artificial_intelligence"

## Response of uncostomised chatbot

Here is response of openai model which was trained on data upto 2021.

In [ ]:
def get_openai_response(prompt):
    initial_answer = openai.Completion.create(
        model=open_ai_model_name,
        prompt=prompt,
        max_tokens=max_tokens
    )["choices"][0]["text"].strip()

    return initial_answer


In [ ]:
prompt_template = """
Question:{}
Answer: {}
"""
for question in QUESTIONS:
    prompt = prompt_template.format(question, "")
    initial_answer = get_openai_response(prompt)
    print(prompt_template.format(question, initial_answer))
    # print('-'*100)


Question:What significant achievement did OpenAI's GPT-3 accomplish compared to Microsoft's Turing Natural Language Generation model?
Answer: OpenAI's GPT-3 (Generative Pre-trained Transformer-3) is a language processing AI model that gained significant attention and recognition for its impressive capabilities. It was released in 2020 and generated a lot of buzz in the field of natural language processing.

The most significant achievement of GPT-3 is its size, with 175 billion parameters, making it the largest language model to date. This is about ten times larger than Microsoft's Turing Natural Language Generation model, which had


Question:When did Google release Gemini 1.5?
Answer: As a language model AI, I don't have access to precise dates but based on my research, Google released Gemini 1.5 in 2019.


Question:When did Apple announced 'Apple Intelligence' which incorporates ChatGPT into new IPhones and Siri?
Answer: Apple announced 'Apple Intelligence', which incorporates Chat

**
As can seen above the chatboot is eithere halucinating or providing I don't have information about the questions, because these questions are based on recent events.**

Lets now collect the most upto date data and provide the relevant data as context for the chatbot to improve the perfromance.

## Step 1: Collect and clean dataset from web

We will download the data from [Timeline of artificial intelligence](https://en.wikipedia.org/wiki/Timeline_of_artificial_intelligence) Wikipedia page

- Part of the the data was created after the model was trained
- Using web scrapping using
- requist and Beautisoup libraries

Applied web scraping to extract table data from a Wikipedia page using several key libraries. The process starts by sending a GET request to the specified URL using the requests library. After verifying that the request is successful, the page content is parsed with BeautifulSoup. The extracted data is stored in pandas DataFrame. This allows the data to be structured for further analysis or manipulation. The web scraping process includes:

- Requests for sending HTTP requests to fetch the webpage.
- BeautifulSoup for parsing and navigating the HTML content.
- Pandas to store and manage the extracted data in a structured DataFrame format.

This setup allows efficient extraction of table data from websites and stores it for further use or analysis.

In [ ]:

# Pandas data frame to hold the data
df = pd.DataFrame(columns=['text'])
# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Iterate through each table and extract data
    for i, table in enumerate(tables):

      # the last table contains and ignore this table
      if len(tables) - 1 == i:
            continue


      # Find all rows in the table
      rows = table.find_all('tr')

      # Extract header (if any)
      headers = [header.get_text(strip=True) for header in rows[0].find_all(['th', 'td'])]
      # print(f"table headers are :{headers}")

      # Extract table data
      for row in rows[1:]:
          cells = row.find_all(['td', 'th'])
          data = [cell.get_text(strip=True) for cell in cells]
          # update data to dataframe
          df.loc[len(df)] = [data]

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


## Data Wrangling

1. Remove rows that contain irrelevant information
2. Remove citation numbers



In [ ]:
def pre_process_data(df):
    """
    Pre-processes the input DataFrame by cleaning and structuring the data extracted from tables.

    Args:
    - df (pd.DataFrame): Input DataFrame with a 'text' column containing data to process.

    Returns:
    - pd.DataFrame: A cleaned and processed DataFrame containing 'date' and 'text' columns.
    """

    # Remove unwanted initial rows from the DataFrame (first 7 rows).
    df = df.iloc[7:, :]

    # Extract the date from the first element of the 'text' if the row has exactly two elements.
    df["date"] = df['text'].map(lambda val: val[0] if len(val) == 2 else None)

    # Extract the main text (last element) from the 'text' column.
    df["text_new"] = df['text'].map(lambda val: val[-1])

    # Remove rows where 'text_new' has fewer than 25 characters.
    threshold = 25
    df['char_len'] = df['text_new'].map(lambda txt: len(txt))
    df = df.loc[df['char_len'] > threshold, :]

    # Print the DataFrame columns for debugging or inspection (optional).
    print(df.columns)

    # Forward-fill missing 'date' values to ensure no missing dates.
    df['date'].ffill(inplace=True)

    # Remove citation numbers in the format [number] from the 'text_new' column using regex.
    df['text_clean'] = df['text_new'].map(lambda txt: re.sub(r"\[\d+\]", "", txt))

    # Combine the 'date' and cleaned text for contextualization (e.g., "Around {date}, {text}").
    df['text_clean'] = df.apply(lambda row: f"Around {row['date']}, {row['text_clean']}", axis=1)

    # Select only the 'date' and cleaned text columns, reset the index, and rename 'text_clean' to 'text'.
    df = df[['date', 'text_clean']].reset_index(drop=True)
    df.rename(columns={'text_clean': 'text'}, inplace=True)

    # Return the cleaned DataFrame.
    return df



In [ ]:
costum_df = pre_process_data(df)
costum_df.head()

Index(['text', 'date', 'text_new', 'char_len'], dtype='object')


<ipython-input-11-5b35493532a1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df['text'].map(lambda val: val[0] if len(val) == 2 else None)
<ipython-input-11-5b35493532a1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_new"] = df['text'].map(lambda val: val[-1])
<ipython-input-11-5b35493532a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,date,text
0,Antiquity,"Around Antiquity, Greek myths ofHephaestusandP..."
1,Antiquity,"Around Antiquity, Sacred mechanical statuesbui..."
2,10th century BC,"Around 10th century BC, Yan Shi presentedKing ..."
3,384 BC–322 BC,"Around 384 BC–322 BC, Aristotledescribed thesy..."
4,3rd century BC,"Around 3rd century BC, Ctesibiusinvents a mech..."


## Compute custom dataset embedding using OpenAI embedding

In [ ]:
def get_embedding(df, text_col):
    """
    Generates embeddings for text data in a specified column of the DataFrame using an OpenAI embedding model.

    This function processes the text in batches and sends it to the OpenAI model to generate embeddings for each
    text entry. The resulting embeddings are added as a new column to the DataFrame.

    Args:
    - df (pd.DataFrame): The input DataFrame containing text data.
    - text_col (str): The name of the column in the DataFrame that contains the text for which embeddings will
                      be generated.

    Returns:
    - pd.DataFrame: The original DataFrame with an additional 'embeddings' column containing the embeddings
                    for each row of text.
    """

    # Initialize an empty list to store generated embeddings for each text entry.
    embeddings = []

    # Process the text data in batches to avoid exceeding token limits or API request limits.
    for i in range(0, len(df), batch_size):
        # Send a batch of text data to the OpenAI embedding model.
        response = openai.Embedding.create(
            input=df.iloc[i:i+batch_size][text_col].tolist(),  # Convert the batch of text to a list format.
            engine=embedding_model_name  # Specify the OpenAI embedding model (e.g., "text-embedding-ada-002").
        )

        # Extract the embeddings from the API response and append them to the 'embeddings' list.
        embeddings.extend([data["embedding"] for data in response["data"]])

    # Add the list of embeddings as a new column ('embeddings') to the original DataFrame.
    df["embeddings"] = embeddings

    # Return the updated DataFrame, which now includes the generated embeddings.
    return df


In [ ]:
costum_df = get_embedding(costum_df, 'text')


In [ ]:
costum_df.head(5)

,date,text,embeddings
0,Antiquity,"Around Antiquity, Greek myths ofHephaestusandP...","[-0.015202178619801998, -0.016426872462034225,..."
1,Antiquity,"Around Antiquity, Sacred mechanical statuesbui...","[-0.008902313187718391, 0.0002474953362252563,..."
2,10th century BC,"Around 10th century BC, Yan Shi presentedKing ...","[-0.0008177312556654215, -0.034276701509952545..."
3,384 BC–322 BC,"Around 384 BC–322 BC, Aristotledescribed thesy...","[0.01567845791578293, 0.020865513011813164, -0..."
4,3rd century BC,"Around 3rd century BC, Ctesibiusinvents a mech...","[-0.018270079046487808, 0.0032011265866458416,..."


In [ ]:
# question_df = pd.DataFrame()
# question_df['question'] =QUESTIONS
# question_df = get_embedding(question_df, 'question')
# question_df

## Find relevant data for each user query

- To find relevant data for each question:
    - Compute embedding of each question
    - Compute embedding of the colleted data (which is done above)
    - Compute similarity in the embedding space using Cosine Similarity
    - Then find relevant data



In [ ]:
def sort_custom_data_by_relevance_to_query(custom_data, query):
    """
    Sorts custom data based on its relevance to a given query using cosine similarity on embeddings.

    This function computes the similarity between a query and the embeddings of the custom data using
    cosine distance. The custom data is then sorted in ascending order of distance, meaning the most
    relevant data (smallest distance) will appear first.

    Args:
    - custom_data (pd.DataFrame): The input DataFrame containing a column 'embeddings', where each entry
                                  is an embedding vector.
    - query (str): The query string for which relevance is measured.

    Returns:
    - pd.DataFrame: The custom data sorted by relevance (smallest cosine distance to the query).
    """

    # Generate the embedding for the query using the OpenAI embedding model.
    query_embedding = openai.Embedding.create(input=query, model=embedding_model_name)

    # Calculate the cosine distance between the query embedding and each of the custom data embeddings.
    distance = distances_from_embeddings(query_embedding.data[0]['embedding'],
                                         custom_data['embeddings'].values,
                                         distance_metric='cosine')

    # Add the computed distances to the custom_data DataFrame as a new column 'distance'.
    custom_data['distance'] = distance

    # Sort the custom_data by the 'distance' column in ascending order, so the most relevant data comes first.
    custom_data = custom_data.sort_values(by='distance', ascending=True).reset_index(drop=True)

    # Return the custom_data sorted by relevance to the query.
    return custom_data


## Create query which contains context

Generates a text prompt by combining the question with relevant context from the custom data. The prompt for an OpenAI Completion model and the model has a limited context window. Thus, the context is included in the prompt until the  total token count reaches the context window limit.

In [ ]:
prompt_template_with_context = """
  Answer the question based on the context below, and if the question
  can not be answered based on the context and previous knowledge, say "I don't know".

  Context:

  {}

  Question:

  {}
  Answer:
  """
print(prompt_template_with_context)


  Answer the question based on the context below, and if the question
  can not be answered based on the context and previous knowledge, say "I don't know".

  Context: 
  
  {}

  Question: 
  
  {}
  Answer:
  


In [ ]:
def create_prompt(question, custom_data, max_token_count):
    """
    Generates a text prompt by combining the question with relevant context from the custom data.

    This function creates a prompt for an OpenAI Completion model by selecting text from the custom
    data that is most relevant to the question. The context is included in the prompt until the
    total token count reaches the specified limit.

    Args:
    - question (str): The question or query for which the prompt is being created.
    - custom_data (pd.DataFrame): A DataFrame containing text data and its embeddings.
    - max_token_count (int): The maximum number of tokens allowed in the prompt to ensure
                             it does not exceed the model's limit.

    Returns:
    - str: A formatted prompt string containing relevant context and the original question.
    """

    # Create a tokenizer to align with the OpenAI model's tokenization, using the 'cl100k_base' encoding.
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Calculate the initial token count by encoding the template and the question.
    current_token_count = len(tokenizer.encode(prompt_template_with_context)) + \
                          len(tokenizer.encode(question))

    # Sort the custom data by relevance to the question using their embeddings.
    relevant_data = sort_custom_data_by_relevance_to_query(custom_data.copy(), question)

    # Initialize an empty list to hold the selected context text.
    context = []

    # Loop through the sorted relevant text data and add them to the context until the max token limit is reached.
    for text in relevant_data["text"].values:

        # Count the number of tokens in the current text row.
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # If adding the current text doesn't exceed the max token count, add it to the context list.
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break  # Stop if adding more text exceeds the token limit.

    # Format the prompt by inserting the context and question into the prompt template.
    return prompt_template_with_context.format("\n".join(context), question)


## Custom Query Completion


In [ ]:
def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=150):
    """
    Answers a given question using context from a dataframe and the OpenAI Completion model.

    This function takes a question and searches the provided dataframe for relevant context to
    build a prompt. It sends the prompt to the OpenAI Completion model and retrieves an answer
    to the question. If an error occurs during model execution, it returns an empty string.

    Args:
    - question (str): The question or query to be answered.
    - df (pd.DataFrame): A DataFrame containing text data and embeddings used to create context
                         for the model's response.
    - max_prompt_tokens (int, optional): The maximum number of tokens allowed in the generated prompt.
                                         Default is 1800 tokens.
    - max_answer_tokens (int, optional): The maximum number of tokens allowed for the model's answer.
                                         Default is 150 tokens.

    Returns:
    - tuple: A tuple containing:
        - prompt (str): The prompt that was sent to the OpenAI Completion model.
        - answer (str): The model's answer to the question. If an error occurs, the answer is an empty string.
    """

    # Generate a prompt that includes relevant context from the DataFrame and the question.
    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        # Send the prompt to the OpenAI Completion model to generate an answer.
        response = openai.Completion.create(
            model=open_ai_model_name,
            prompt=prompt,             # The prompt containing context and the question.
            max_tokens=max_answer_tokens  # Limit the length of the generated answer.
        )

        # Return the prompt and the model's response text.
        return prompt, response["choices"][0]["text"].strip()

    except Exception as e:
        # If an error occurs, print the error and return the prompt with an empty answer.
        print(e)
        return prompt, ""


## Custom chatbot performance demonstration

This section demonstrates the chatbot that uses RAG as a long term memmory provides contextually relevant and acurate response compared with the chatbot without RAG.

In [ ]:
for i, question in enumerate(QUESTIONS):
    prompt = prompt_template.format(question, "")
    initial_answer = get_openai_response(prompt)

    _, final_answer = answer_question(question=question,
                              df=costum_df)
    print(f"Question {i+1}: {question}\n")
    print(f"Inital answer (without context): {initial_answer}\n")
    print(f"Final answer (with context): {final_answer}")
    print('********')

Question 1: What significant achievement did OpenAI's GPT-3 accomplish compared to Microsoft's Turing Natural Language Generation model?

Inital answer (without context): OpenAI's GPT-3 (Generative Pre-trained Transformer 3) achieved a groundbreaking feat by being able to generate coherent and convincing human-like text with minimal prompts or input, while Microsoft's Turing Natural Language Generation model (T-NLG) mainly focused on more specific tasks and applications, such as summarizing texts or answering questions. GPT-3 is also significantly larger, with 175 billion parameters compared to T-NLG's 17 billion. This allows GPT-3 to have

Final answer (with context): OpenAI's GPT-3 had a capacity ten times greater than that of Microsoft's Turing Natural Language Generation model.
********
Question 2: When did Google release Gemini 1.5?

Inital answer (without context): Google released Gemini 1.5 on January 25, 2016.

Final answer (with context): Around February 15, 2024.
********
Que

## Explanation

**Question 1:** What significant achievement did OpenAI's GPT-3 accomplish compared to Microsoft's Turing Natural Language Generation model?

Inital answer (without context): OpenAI's GPT-3 (Generative Pre-trained Transformer 3) achieved a groundbreaking feat by being able to generate coherent and convincing human-like text with minimal prompts or input, while Microsoft's Turing Natural Language Generation model (T-NLG) mainly focused on more specific tasks and applications, such as summarizing texts or answering questions. GPT-3 is also significantly larger, with 175 billion parameters compared to T-NLG's 17 billion. This allows GPT-3 to have

Final answer (with context): OpenAI's GPT-3 had a capacity ten times greater than that of Microsoft's Turing Natural Language Generation model.

**Review: Both custum and naive chatbots have got a sensible answer.**
********
**Question 2:** When did Google release Gemini 1.5?

Inital answer (without context): Google released Gemini 1.5 on January 25, 2016.

Final answer (with context): Around February 15, 2024.

**Review: The custum chatbot got the answer correct, but the naive chatbot got it wrong.**
********
**Question 3:** When did Apple announced 'Apple Intelligence' which incorporates ChatGPT into new IPhones and Siri?

Inital answer (without context): There is no official announcement from Apple regarding a feature called "Apple Intelligence" or the incorporation of ChatGPT into new iPhones and Siri. Apple has not made any public statements about using ChatGPT or any other AI technology in iPhones or Siri.

Final answer (with context): Around 2024, on June 10, 2024.

**Review: The custum got it correct, but not the naive chatbot.**
********
**Question 4:** What was the purpose of the inaugural 'AI Insight Forum' held by the US Senate in September 2023, and who were some of the prominent attendees?

Inital answer (without context): The purpose of the inaugural 'AI Insight Forum' held by the US Senate in September 2023 was to bring together lawmakers, industry leaders, and experts to discuss the impact and future of artificial intelligence (AI) on society and the economy. The forum aimed to provide insights on current and potential challenges and opportunities posed by AI and to foster collaboration and informed decision-making among policymakers.

Some of the prominent attendees at the AI Insight Forum included Senate leaders such as Majority Leader Chuck Schumer and Minority Leader Mitch McConnell

Final answer (with context): The purpose of the inaugural 'AI Insight Forum' was to familiarize senators with the nature of AI and its risks, and to discuss needed safeguards and legislation. Prominent attendees included senators, CEOs, civil rights leaders, and industry representatives, such as Senate Majority Leader Chuck Schumer, U.S. Senator Martin Heinrich, Elon Musk, Mark Zuckerberg, Sam Altman, Sundar Pichai, Bill Gates, Satya Nadella, Jensen Huang, Arvind Krishna, Alex Karp, Charles Rivkin, Meredith Stiehm, Liz Shuler, and Maya Wiley. Unfortunately, we are unable to provide a more comprehensive list of attendees without further information about the forum.

**Review: The naive chatbot gave a general answer while the custum chabot gave correct answer.**

********
**Question 5:** What were the key developments and announcements related to AI made in February 2024 by Google and OpenAI?

Inital answer (without context): There is no way to accurately answer this question as February 2024 has not occurred yet and we cannot predict the future development of AI by Google and OpenAI. It is likely that there will be ongoing advancements and announcements in the field of AI by these companies, but it is not possible to provide specific information about them at this time.

Final answer (with context): In addition to the release of Sora, a text-to-video model, by OpenAI on February 15, 2024, Google also released Gemini 1.5 in limited beta with a context length of up to 1 million tokens on February 15, 2024, and Apple announced the incorporation of ChatGPT into new iPhones and Siri on June 10, 2024.

**Review: The custum chatbot got it right, but naive chatbots was not able to  answer the question.**
********

## Chatbot for interactive user query



In [ ]:
# to end the conversation, type stop
while True:
  question = input("Enter question (type 'stop' to quit):")
  if question == "stop":
    break
  prompt = prompt_template.format(question, "")
  initial_answer = get_openai_response(prompt)

  _, final_answer = answer_question(question=question,
                            df=costum_df)
  print(f"Question: {question}\n")
  print(f"Inital answer (without context): {initial_answer}\n")
  print(f"Final answer (with context): {final_answer}")
  print('********\n')

  # print(user_input)


Enter question (type 'stop' to quit):What is AI?
Question 5: What is AI?

Inital answer (without context): AI stands for Artificial Intelligence, which refers to the simulation of human intelligence in computer systems. It involves creating intelligent machines that can think, learn, and perform tasks that typically require human intelligence, such as problem-solving, decision making, and language translation. Artificial Intelligence is a broad field that encompasses various subfields, such as Machine Learning, Natural Language Processing, and Robotics. It aims to make computers or machines capable of performing tasks that usually require human intelligence, making them more efficient and effective in performing complex

Final answer (with context): AI, or artificial intelligence, is a broad field of computer science focused on creating intelligent machines that can replicate human behavior, solve problems, and make decisions.
********
Enter question (type 'stop' to quit):Who coined th

## Conclusion

Here we showed that RAG minimises the hallucination of LLM model by providing relevant context!